## 1. Import Dependencies

In [1]:
# import dependencies
import pandas as pd
import requests
from config import tcgapikey, pgpassword, aws_endpoint
import datetime
from sqlalchemy import create_engine, dialects
!pip3 install psycopg2-binary
from re import search


You should consider upgrading via the 'c:\users\ianfa\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


## 2. Perform API Call to get raw data

In [2]:
# initial API call
url = 'https://api.pokemontcg.io/v2/cards?q=set.series:Sword%20&%20Shield&page=1'

In [3]:
# get request
data = requests.get(url, headers={'X-Api-Key':tcgapikey}).json()

In [4]:
# loop to get all pages of query
query_results = True
i = 2
while query_results:
    url = 'https://api.pokemontcg.io/v2/cards?q=set.series:Sword%20&%20Shield&page='+str(i)
    card_info = requests.get(url, headers={'X-Api-Key':tcgapikey}).json()
    data['data'] = data['data']+card_info['data']
    data['count'] = data['count']+card_info['count']
    print(f'Appended page {i}')
    i += 1
    if card_info['count']==card_info['pageSize']: 
        continue
    else:
        query_results= False
del data['page']
del data['pageSize']


Appended page 2
Appended page 3
Appended page 4
Appended page 5
Appended page 6
Appended page 7


## 3. Examine data

In [5]:
# investigate columns
pd.DataFrame(data['data']).columns

Index(['id', 'name', 'supertype', 'subtypes', 'hp', 'types', 'rules',
       'attacks', 'weaknesses', 'retreatCost', 'convertedRetreatCost', 'set',
       'number', 'artist', 'rarity', 'nationalPokedexNumbers', 'legalities',
       'images', 'tcgplayer', 'evolvesTo', 'flavorText', 'evolvesFrom',
       'abilities', 'resistances'],
      dtype='object')

In [6]:
# see dataframe
pd.DataFrame(data['data']).head(5)

,id,name,supertype,subtypes,hp,types,rules,attacks,weaknesses,retreatCost,...,rarity,nationalPokedexNumbers,legalities,images,tcgplayer,evolvesTo,flavorText,evolvesFrom,abilities,resistances
0,swsh35-1,Venusaur V,Pokémon,"[Basic, V]",220,[Grass],"[V rule: When your Pokémon V is Knocked Out, y...","[{'name': 'Pollen Bomb', 'cost': ['Grass', 'Gr...","[{'type': 'Fire', 'value': '×2'}]","[Colorless, Colorless, Colorless]",...,Rare Holo V,[3],"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,NaN,NaN,NaN,NaN,NaN
1,swsh35-2,Weedle,Pokémon,[Basic],60,[Grass],NaN,"[{'name': 'Call for Family', 'cost': ['Colorle...","[{'type': 'Fire', 'value': '×2'}]",[Colorless],...,Common,[13],"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,[Kakuna],Its poison stinger is very powerful. Its brigh...,NaN,NaN,NaN
2,swsh1-1,Celebi V,Pokémon,"[Basic, V]",180,[Grass],"[V rule: When your Pokémon V is Knocked Out, y...","[{'name': 'Find a Friend', 'cost': ['Grass'], ...","[{'type': 'Fire', 'value': '×2'}]",[Colorless],...,Rare Holo V,[251],"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh1/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,NaN,NaN,NaN,NaN,NaN
3,swsh4-1,Weedle,Pokémon,[Basic],40,[Grass],NaN,"[{'name': 'Bug Hunch', 'cost': ['Grass'], 'con...","[{'type': 'Fire', 'value': '×2'}]",[Colorless],...,Common,[13],"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh4/...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,[Kakuna],Its poison stinger is very powerful. Its brigh...,NaN,NaN,NaN
4,swsh35-3,Kakuna,Pokémon,[Stage 1],80,[Grass],NaN,"[{'name': 'Bug Bite', 'cost': ['Grass'], 'conv...","[{'type': 'Fire', 'value': '×2'}]","[Colorless, Colorless, Colorless]",...,Common,[14],"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,{'url': 'https://prices.pokemontcg.io/tcgplaye...,[Beedrill],"While awaiting evolution, it hides from predat...",Weedle,NaN,NaN


In [7]:
# examine set information breakdown
pd.DataFrame(data['data'])['set'][0]

{'id': 'swsh35',
 'name': "Champion's Path",
 'series': 'Sword & Shield',
 'printedTotal': 73,
 'total': 80,
 'legalities': {'unlimited': 'Legal',
  'standard': 'Legal',
  'expanded': 'Legal'},
 'ptcgoCode': 'CPA',
 'releaseDate': '2020/09/25',
 'updatedAt': '2020/10/25 13:45:00',
 'images': {'symbol': 'https://images.pokemontcg.io/swsh35/symbol.png',
  'logo': 'https://images.pokemontcg.io/swsh35/logo.png'}}

In [8]:
pd.DataFrame(data['data'])['tcgplayer'][0]

{'url': 'https://prices.pokemontcg.io/tcgplayer/swsh35-1',
 'updatedAt': '2021/06/28',
 'prices': {'holofoil': {'low': 1.25,
   'mid': 2.44,
   'high': 19.95,
   'market': 2.04,
   'directLow': 3.98}}}

## 4. Clean Data

In [9]:
# for each entry, split data from set dictionary
for i in range(len(data['data'])):
    try:
        data['data'][i]['prices']=data['data'][i]['tcgplayer']['prices']
    except KeyError:
        data['data'][i]['prices'] = None
    
    try: 
    #remove direct market; no direct market exists for newest cards so pass
        for card_style in data['data'][i]['prices']:
            del data['data'][i]['prices'][card_style]['directLow']
    except TypeError:
        pass



# set info
    #set_id
    try:    
        data['data'][i]['set_id']=data['data'][i]['set']['id']
    except KeyError:
        data['data'][i]['set_id']= None

    #set_name
    try:    
        data['data'][i]['set_name']=data['data'][i]['set']['name']
    except KeyError:
        data['data'][i]['set_name']= None

    #add timestamp
    data['data'][i]['date']= datetime.datetime.now()

    # collector data
    if search ('\D', data['data'][i]['number']):
        data['data'][i]['number'] = data['data'][i]['number']
    else:
        data['data'][i]['number'] = str(data['data'][i]['number']).zfill(3) + "/" + str(data['data'][i]['set']['printedTotal']).zfill(3)




In [10]:
# create data frame & drop unneeded information
data_clean_df = pd.DataFrame(data['data']).drop(columns=['supertype', 'subtypes', 'hp', 'rules', 'attacks', 'weaknesses','retreatCost', 'convertedRetreatCost','artist', 'rarity', 'nationalPokedexNumbers','evolvesTo', 'flavorText', 'evolvesFrom', 'abilities', 'resistances','types', 'tcgplayer', 'set'])

## 5. Examine cleaned data

In [11]:
data_clean_df.head(5)

,id,name,number,legalities,images,prices,set_id,set_name,date
0,swsh35-1,Venusaur V,001/073,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,"{'holofoil': {'low': 1.25, 'mid': 2.44, 'high'...",swsh35,Champion's Path,2021-06-30 20:23:15.888773
1,swsh35-2,Weedle,002/073,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,"{'reverseHolofoil': {'low': 0.09, 'mid': 0.26,...",swsh35,Champion's Path,2021-06-30 20:23:15.888773
2,swsh1-1,Celebi V,001/202,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh1/...,"{'holofoil': {'low': 1.25, 'mid': 2.47, 'high'...",swsh1,Sword & Shield,2021-06-30 20:23:15.888773
3,swsh4-1,Weedle,001/185,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh4/...,"{'normal': {'low': 0.02, 'mid': 0.15, 'high': ...",swsh4,Vivid Voltage,2021-06-30 20:23:15.888773
4,swsh35-3,Kakuna,003/073,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,"{'normal': {'low': 0.01, 'mid': 0.16, 'high': ...",swsh35,Champion's Path,2021-06-30 20:23:15.888773


In [12]:
data_clean_df.rename(columns={'number':'collector_number'})

,id,name,collector_number,legalities,images,prices,set_id,set_name,date
0,swsh35-1,Venusaur V,001/073,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,"{'holofoil': {'low': 1.25, 'mid': 2.44, 'high'...",swsh35,Champion's Path,2021-06-30 20:23:15.888773
1,swsh35-2,Weedle,002/073,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,"{'reverseHolofoil': {'low': 0.09, 'mid': 0.26,...",swsh35,Champion's Path,2021-06-30 20:23:15.888773
2,swsh1-1,Celebi V,001/202,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh1/...,"{'holofoil': {'low': 1.25, 'mid': 2.47, 'high'...",swsh1,Sword & Shield,2021-06-30 20:23:15.888773
3,swsh4-1,Weedle,001/185,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh4/...,"{'normal': {'low': 0.02, 'mid': 0.15, 'high': ...",swsh4,Vivid Voltage,2021-06-30 20:23:15.888773
4,swsh35-3,Kakuna,003/073,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh35...,"{'normal': {'low': 0.01, 'mid': 0.16, 'high': ...",swsh35,Champion's Path,2021-06-30 20:23:15.888773
...,...,...,...,...,...,...,...,...,...
1632,swsh6-229,Urn of Vitality,229/198,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh6/...,"{'holofoil': {'low': 7.5, 'mid': 11.86, 'high'...",swsh6,Chilling Reign,2021-06-30 20:23:15.897556
1633,swsh6-230,Welcoming Lantern,230/198,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh6/...,"{'holofoil': {'low': 7.0, 'mid': 9.79, 'high':...",swsh6,Chilling Reign,2021-06-30 20:23:15.897556
1634,swsh6-231,Water Energy,231/198,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh6/...,"{'holofoil': {'low': 25.0, 'mid': 34.0, 'high'...",swsh6,Chilling Reign,2021-06-30 20:23:15.897556
1635,swsh6-232,Psychic Energy,232/198,"{'unlimited': 'Legal', 'standard': 'Legal', 'e...",{'small': 'https://images.pokemontcg.io/swsh6/...,"{'holofoil': {'low': 24.0, 'mid': 32.42, 'high...",swsh6,Chilling Reign,2021-06-30 20:23:15.897556


In [13]:
data_clean_df['legalities'][0]

{'unlimited': 'Legal', 'standard': 'Legal', 'expanded': 'Legal'}

In [22]:
data_clean_df['images'][1636]

{'small': 'https://images.pokemontcg.io/swsh6/233.png',
 'large': 'https://images.pokemontcg.io/swsh6/233_hires.png'}

In [23]:
data_clean_df['prices'][1636]

{'holofoil': {'low': 22.0, 'mid': 25.0, 'high': 50.0, 'market': 25.74}}

In [16]:
# Sample individual data set
data_clean_df.loc[1,:]

id                                                     swsh35-2
name                                                     Weedle
number                                                  002/073
legalities    {'unlimited': 'Legal', 'standard': 'Legal', 'e...
images        {'small': 'https://images.pokemontcg.io/swsh35...
prices        {'reverseHolofoil': {'low': 0.09, 'mid': 0.26,...
set_id                                                   swsh35
set_name                                        Champion's Path
date                                 2021-06-30 20:23:15.888773
Name: 1, dtype: object

## 6. Load into database

In [17]:
# link to the database & create engine
engine = create_engine(f'postgresql://postgres:{pgpassword}@{aws_endpoint}:5432/postgres', echo=False)

In [18]:
# load into database with format
data_clean_df.to_sql(
    'SwShSeries',
    con=engine,
    if_exists='append',
    index=False,
    dtype={
    'legalities':dialects.postgresql.JSON,
    'images':dialects.postgresql.JSON,
    'prices':dialects.postgresql.JSON}
    )

In [19]:
# export as csv for local test
data_clean_df.to_csv(f'SwShSeries_{datetime.date.today()}.csv', index = False)

In [20]:
# export one record for local test
data_clean_df.loc[1,:].to_json('data.js')